In [99]:
import pandas as pd
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, Dict, Iterable, List, Tuple
import csv
import io
import math

In [89]:
# ---------- Utilitaires ----------

def fr_float(s: str | None) -> Optional[float]:
    """Convertit une chaîne avec virgule en float (ou None si vide)."""
    if s is None:
        return None
    s = s.strip().strip('"').strip("'")
    if s == "" or s.lower() in {"nan", "null"}:
        return None
    # Remplacer la virgule décimale par un point
    return float(s.replace(",", "."))


Définition du mur

In [90]:
# ---------- Modèle de données ----------
@dataclass(frozen=True)
class Mur:
    """Représente un mur avec ses coordonnées/mesures."""
    id: str
    x: Optional[float]
    y: Optional[float]
    z: Optional[float]
    centre_x: Optional[float]
    centre_y: Optional[float]
    centre_z: Optional[float]

    @staticmethod
    def from_csv_row(row: dict) -> "Mur":
        """Crée un Mur depuis une ligne CSV (clés: Murs, X, Y, Z)."""
        return Mur(
            id=row.get("Murs", "").strip(),
            x=fr_float(row.get("X")),
            y=fr_float(row.get("Y")),
            z=fr_float(row.get("Z")),
            centre_x=fr_float(row.get("centre_x")),
            centre_y=fr_float(row.get("centre_y")),
            centre_z=fr_float(row.get("centre_z")),
        )

Chargement des murs et frontieres

In [91]:
# ---------- Parsing du CSV ----------

def lire_murs_csv(source: str | io.TextIOBase) -> Dict[str, Mur]:
    """
    Lit un CSV et renvoie un dict {id_mur: Mur}.
    Accepte des valeurs X/Y/Z avec virgule comme décimale.
    """
    stream = open(source, "r", encoding="utf-8")

    reader = csv.DictReader(stream)
    murs: Dict[str, Mur] = {}
    for row in reader:
        mur = Mur.from_csv_row(row)
        if not mur.id:
            # Ignore les lignes sans identifiant
            continue
        murs[mur.id] = mur
    return murs

In [92]:
# ---------- Exemple d’utilisation ----------

murs = lire_murs_csv("./data/murs.csv")

print(murs)                       


{'0AX0': Mur(id='0AX0', x=4.0, y=0.15, z=3.0, centre_x=None, centre_y=None, centre_z=None), '0AX1': Mur(id='0AX1', x=4.0, y=0.15, z=3.0, centre_x=None, centre_y=None, centre_z=None), '0AY0': Mur(id='0AY0', x=0.15, y=3.0, z=3.0, centre_x=None, centre_y=None, centre_z=None), '0AY1': Mur(id='0AY1', x=0.15, y=3.0, z=3.0, centre_x=None, centre_y=None, centre_z=None), '0BX0': Mur(id='0BX0', x=4.0, y=0.15, z=3.0, centre_x=None, centre_y=None, centre_z=None), '0BX1': Mur(id='0BX1', x=4.0, y=0.15, z=3.0, centre_x=None, centre_y=None, centre_z=None), '0BY0': Mur(id='0BY0', x=0.15, y=3.0, z=3.0, centre_x=None, centre_y=None, centre_z=None), '0BY1': Mur(id='0BY1', x=0.15, y=3.0, z=3.0, centre_x=None, centre_y=None, centre_z=None), '0CX0': Mur(id='0CX0', x=4.0, y=0.15, z=3.0, centre_x=None, centre_y=None, centre_z=None), '0CX1': Mur(id='0CX1', x=4.0, y=0.15, z=3.0, centre_x=None, centre_y=None, centre_z=None), '0CY0': Mur(id='0CY0', x=0.15, y=3.0, z=3.0, centre_x=None, centre_y=None, centre_z=None)

In [ ]:
def lire_frontieres_csv(source: str | io.TextIOBase) -> List[Tuple[str, str]]:
    """
    Lit un CSV de frontières (colonnes: Mur1, Mur2)
    et renvoie une liste de couples (mur1, mur2) uniques.
    """
    stream = open(source, "r", encoding="utf-8")
    reader = csv.DictReader(stream)
    frontieres_set = set()  # pour éviter les doublons

    for row in reader:
        m1 = row.get("Mur1", "").strip()
        m2 = row.get("Mur2", "").strip()
        if not m1 or not m2:
            continue

        # Tri pour éviter les inversions (("A","B") == ("B","A"))
        pair = tuple(sorted((m1, m2)))
        frontieres_set.add(pair)

    # Ferme le fichier si on l’a ouvert ici
    if isinstance(source, (str, bytes)):
        stream.close()

    # Convertit l’ensemble en liste triée
    return frontieres_set


Trouver la longueur et largeur

In [94]:
def frontiere_nord(piece, frontieres):
    target = "0"+piece+"X1"
    for mur1, mur2 in frontieres:
        if mur1 == target :
            return mur2[1:-2]
        if mur2 == target :
            return mur1[1:-2]
    return False 

In [95]:
def longueur_totale_y(murs, frontieres):
    l_tot = 0
    piece_select = "DEPART"

    while piece_select != False :
        piece_select = frontiere_nord(piece_select, frontieres)
        if piece_select !=False :
            l_tot += ( murs["0" + piece_select + "X0"].y ) + murs["0" + piece_select + "Y0"].y + ( murs["0" + piece_select + "X1"].y )   #Le mur + l'épaisseur des murs en travers
    return round(l_tot, 3)

In [96]:
def frontiere_ouest(piece, frontieres):
    target = "0"+piece+"Y0"
    for mur1, mur2 in frontieres:
        if mur1 == target :
            return mur2[1:-2]
        if mur2 == target :
            return mur1[1:-2]
    return False 

In [97]:
def largeur_totale_x(murs, frontieres):
    l_tot = 0
    piece_select = "DEPART"

    while piece_select != False :
        piece_select = frontiere_ouest(piece_select, frontieres)
        if piece_select !=False :
            l_tot += ( murs["0" + piece_select + "Y0"].x ) + murs["0" + piece_select + "X0"].x + ( murs["0" + piece_select + "Y1"].x )   #Le mur + l'épaisseur des murs en travers
    return round(l_tot, 3) 

In [98]:
murs = lire_murs_csv("./data/murs.csv")
frontieres = lire_frontieres_csv("./data/frontiere.csv")

print(longueur_totale_y(murs,frontieres))      
print(largeur_totale_x(murs,frontieres))              

6.6
8.6
